In [10]:
import time

from helper_loader import *
from histalign.backend.registration.alignment import (
    ALIGNMENT_VOLUMES_CACHE_DIRECTORY,
    _module_logger,
    generate_hash_from_targets,
)

set_log_level(500)

In [21]:
def build_alignment_volume(
    alignment_directory: str | Path,
    use_cache: bool = True,
    return_raw_array: bool = False,
    kernel: str = "shepard",
    radius: int = 1,
) -> np.ndarray | vedo.Volume:
    if isinstance(alignment_directory, str):
        alignment_directory = Path(alignment_directory)

    targets = gather_alignment_paths(alignment_directory)

    targets_hash = generate_hash_from_targets(targets)
    cache_path = ALIGNMENT_VOLUMES_CACHE_DIRECTORY / f"{targets_hash}.npz"
    if cache_path.exists() and use_cache:
        _module_logger.debug("Found cached volume. Loading from file.")

        volume = vedo.Volume(np.load(cache_path)["array"])
        if return_raw_array:
            return volume.tonumpy()
        return volume

    with open(targets[0]) as handle:
        temp_settings = AlignmentSettings(**json.load(handle))
    reference_shape = temp_settings.volume_settings.shape

    alignment_volume = vedo.Volume(np.zeros(shape=reference_shape, dtype=np.uint16))
    alignment_array = alignment_volume.tonumpy()
    slicer = VolumeSlicer(volume=alignment_volume)

    _module_logger.debug(f"Generating {len(targets)} meshes.")
    plane_meshes = []
    for index, target in enumerate(targets):
        if index > 0 and index % 5 == 0:
            _module_logger.debug(f"Generated {index} meshes.")

        alignment_settings = load_alignment_settings(target)

        histology_slice = load_image(alignment_settings.histology_path)

        registrator = Registrator(True, True)
        registered_slice = registrator.get_forwarded_image(
            histology_slice, alignment_settings
        )

        plane_mesh = slicer.slice(alignment_settings.volume_settings, return_mesh=True)

        registered_slice = registered_slice[
            plane_mesh.metadata["i_padding"][0] : registered_slice.shape[0]
            - plane_mesh.metadata["i_padding"][1],
            plane_mesh.metadata["j_padding"][0] : registered_slice.shape[1]
            - plane_mesh.metadata["j_padding"][1],
        ]

        plane_mesh.pointdata["ImageScalars"] = registered_slice.flatten()

        plane_meshes.append(plane_mesh)
    _module_logger.debug(f"Generated all {len(targets)} meshes.")

    _module_logger.debug(f"Gathering {len(targets)} meshes into alignment volume.")
    for index, mesh in enumerate(plane_meshes):
        loop_start_time = time.perf_counter()
        if index > 0 and index % 5 == 0:
            _module_logger.debug(f"Gathered {index} meshes.")

        temp_volume = vedo.Volume(np.zeros_like(alignment_array))

        interpolate_start_time = time.perf_counter()
        temp_volume.interpolate_data_from(mesh, kernel=kernel, radius=radius)
        print(
            f"Interpolation ({kernel} - {radius}): {perf_counter() - interpolate_start_time:.2f} seconds."
        )

        temp_array = temp_volume.tonumpy()
        temp_array = np.round(temp_array).astype(np.uint16)

        alignment_array[:] = np.maximum(alignment_array, temp_array)
    alignment_volume.modified()
    _module_logger.debug(f"Gathered all {len(plane_meshes)} meshes.")

    if use_cache:
        _module_logger.debug("Caching volume to file.")
        os.makedirs(ALIGNMENT_VOLUMES_CACHE_DIRECTORY, exist_ok=True)
        np.savez_compressed(cache_path, array=alignment_volume.tonumpy())

    if return_raw_array:
        return alignment_volume.tonumpy()

    return alignment_volume

In [22]:
alignment_path = Path(
    "/home/ediun/git/histalign/projects/project_cortical_depth/93e6cae680"
)

radii = [1, 5, 10]
shepards = [
    build_alignment_volume(
        alignment_path, use_cache=False, kernel="shepard", radius=radius
    )
    for radius in radii
]
gaussians = [
    build_alignment_volume(
        alignment_path, use_cache=False, kernel="gaussian", radius=radius
    )
    for radius in radii
]
linears = [
    build_alignment_volume(
        alignment_path, use_cache=False, kernel="linear", radius=radius
    )
    for radius in radii
]

Interpolation (shepard - 1): 3.76 seconds.
Interpolation (shepard - 5): 12.06 seconds.
Interpolation (shepard - 10): 35.69 seconds.
Interpolation (gaussian - 1): 3.72 seconds.
Interpolation (gaussian - 5): 12.23 seconds.
Interpolation (gaussian - 10): 38.17 seconds.
Interpolation (linear - 1): 3.71 seconds.
Interpolation (linear - 5): 11.68 seconds.
Interpolation (linear - 10): 33.59 seconds.


In [23]:
for i in range(len(radii)):
    shepards[i].cmap(**get_cmap(shepards[i]))
    gaussians[i].cmap(**get_cmap(gausians[i]))
    linears[i].cmap(**get_cmap(linears[i]))

In [27]:
show(
    [
        *shepards,
        *gaussians,
        *linears,
    ],
    n=9,
)

In [35]:
print("Shepards vs gaussians")
print(np.mean(np.abs(shepards[0].tonumpy() - gaussians[0].tonumpy())))
print(np.mean(np.abs(shepards[1].tonumpy() - gaussians[1].tonumpy())))
print(np.mean(np.abs(shepards[2].tonumpy() - gaussians[2].tonumpy())))
print("Shepards vs linears")
print(np.mean(np.abs(shepards[0].tonumpy() - linears[0].tonumpy())))
print(np.mean(np.abs(shepards[1].tonumpy() - linears[1].tonumpy())))
print(np.mean(np.abs(shepards[2].tonumpy() - linears[2].tonumpy())))
print("Gaussians vs linears")
print(np.mean(np.abs(gaussians[0].tonumpy() - linears[0].tonumpy())))
print(np.mean(np.abs(gaussians[1].tonumpy() - linears[1].tonumpy())))
print(np.mean(np.abs(gaussians[2].tonumpy() - linears[2].tonumpy())))

Shepards vs gaussians
51.379307388232654
477.7748197824254
1007.2458990345477
Shepards vs linears
49.621995292148455
512.5643296269645
1042.4910397664971
Gaussians vs linears
50.07724399110347
519.3232684835609
1057.228341157255


In [61]:
slice_ = (slice(305, 306), slice(200, 201), slice(100, 101))

print(shepards[0].tonumpy()[slice_])
print(gaussians[0].tonumpy()[slice_])
print(linears[0].tonumpy()[slice_])

[[[10636]]]
[[[10633]]]
[[[10800]]]
